In [1]:

import os
import warnings
import pandas as pd       
import requests
import mlflow

from automlwrapper import AutoMLWrapper
from sedarapi import SedarAPI


2023-12-03 19:01:09.251894: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-03 19:01:09.302438: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-03 19:01:10.096045: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


In [2]:
experiment_id = 8
remote_server_uri = "http://192.168.220.107:6798"
mlflow.set_tracking_uri(remote_server_uri) 
mlflow.set_experiment(experiment_id=experiment_id)

<Experiment: artifact_location='hdfs://192.168.220.107:9870/datalake/mlruns/8', creation_time=1701619339116, experiment_id='8', last_update_time=1701619339116, lifecycle_stage='active', name='test', tags={'type': 'manual', 'workspace_id': '13b4787c3e454649aa05a4cd680edc37'}>

In [3]:

base_url = "http://192.168.220.107:5000"
email = "admin"
password = "admin"

sedar = SedarAPI(base_url)
sedar.connection.logger.setLevel("INFO") 

sedar.login(email, password)

ws = sedar.get_workspace("13b4787c3e454649aa05a4cd680edc37") 
dataset = ws.get_dataset("986f2e837ca44f3e8c0ee7d2dc0c4287")

query_string = f"SELECT * FROM {dataset.id}"
try:
    query_result = dataset.query_sourcedata(query_string)
    glass_schema = pd.DataFrame.from_dict(query_result['body'])
except Exception as e:
    print(e)


INFO:SedarAPI-Logger:Login successful
INFO:SedarAPI-Logger:The query 'SELECT * FROM 986f2e837ca44f3e8c0ee7d2dc0c4287' for Dataset '986f2e837ca44f3e8c0ee7d2dc0c4287' has been executed successfully.


In [4]:

wrapper = AutoMLWrapper('autogluon')
wrapper.SetOutputDirectory('./out/agl/2')
wrapper.Train(
    data=glass_schema,
    target_column='Type',
    task_type='classification',
    data_type='tabular',
    problem_type='multiclass',
    hyperparameters={'time_limit' : 60*10}
)


No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "./out/agl/2"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.8.8
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #183-Ubuntu SMP Mon Oct 2 11:28:33 UTC 2023
CPU Count:          104
Memory Avail:       707.63 GB / 754.52 GB (93.8%)
Disk Space Avail:   333

In [6]:
wrapper.MlflowUploadTopN(10, {})


2023/12/03 19:01:53 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp4vgvyf__/model, flavor: python_function), fall back to return ['cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.
/usr/local/automl/lib/python3.8/site-packages/mlflow/store/artifact/hdfs_artifact_repo.py:187: FutureWarning: pyarrow.hdfs.connect is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  connected = pa.hdfs.connect(


FileNotFoundError: [Errno 2] No such file or directory: 'hadoop'

In [ ]:

    data = {
    "notebook_id":"b7d9b6f3-5f33-43c8-b9de-a206c761919f",
    "run_id":run_id,
    "experiment_id":experiment_id
   }
res = requests.post(url = "http://192.168.220.107:5000/api/v1/workspaces/13b4787c3e454649aa05a4cd680edc37/add-mlrun/986f2e837ca44f3e8c0ee7d2dc0c4287", data=data)    


In [7]:
import mlflow
logged_model = 'runs:/9d174287efa44683acb1054511af02b0/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(data))

OSError: No such file or directory: '/home/mibbels/sedar-masterarbeit/sedar/sedar/automl/tmp/artifacts/9d174287efa44683acb1054511af02b0/artifacts/model'